In [1]:
# Import gurobi and numpy
from gurobipy import *
import numpy as np
import pandas as pd

## LOAD DATA

In [2]:
demand = pd.read_csv('demand.csv')
machine_dtls = pd.read_csv('machine_dtls.csv')
capacity = pd.read_csv('pathway_compliments.csv')
sector_compliments = pd.read_csv('sector_compliments.csv')

In [3]:
capacity.Adj_Capacity_Daily_Per_Machine = capacity.Adj_Capacity_Daily_Per_Machine.replace(',', '', regex=True)
capacity.Adj_Capacity_Daily_Per_Machine = pd.to_numeric(capacity.Adj_Capacity_Daily_Per_Machine)

In [4]:
sector_compliments['Inverse_Compliment'] = 1/sector_compliments['Compliment']
demand_pw_exists = demand[~demand['PW'].isnull()].reset_index(drop=True)
machine_exists = machine_dtls[~machine_dtls['Work_Sector'].isnull()].reset_index(drop=True)

capacity_needed = capacity[capacity['Pathway'].isin(demand_pw_exists["PW"].unique())]
capacity_needed = capacity_needed[~capacity_needed['Work_Sector'].isnull()].reset_index(drop=True)

In [5]:
# choose which demand subset
demand_col = 'Past_Due_and_Current'

## REFERENCE DICTIONARIES

In [6]:
# Define model and parameters. 
mod1 = Model()

def flatten(t):
    return [item for sublist in t for item in sublist]

sectors = machine_dtls[~machine_dtls['Work_Sector'].isna()]['Work_Sector'].unique()
pathways = capacity_needed["Pathway"].unique()

# Variable Identifiers
days = range(30)
machine_count = range(len(machine_exists))
pw_count = range(len(pathways))
sectors_num = range(int(sectors.max()))
max_operations = range(capacity_needed['Operation_Sequence'].max())

# Machine Dictionary
machine_dic = {}

for i in machine_count:
    machine_dic[i] = machine_exists.iloc[i,0]

# Set pw to numbers
pw_indexing_dic = {}

for p in pw_count:
    pw_indexing_dic[p] = pathways[p]

# PW Dictionary
pw_dic = {} 

for p in pw_count:
    pw_dic[p] = {}
    for o in capacity_needed[capacity_needed['Pathway'] == pathways[p]]['Operation_Sequence'].unique():
        pw_dic[p][o] = list(capacity_needed[(capacity_needed['Pathway'] == pathways[p]) & (capacity_needed['Operation_Sequence'] == o)]['Work_Sector'])

# Capacity Dictionary
capacity_dic = {} 

for p in pw_count:
    capacity_dic[p] = {}

    for w in capacity_needed[capacity_needed['Pathway'] == pathways[p]]['Work_Sector'].unique():
        capacity_dic[p][w] = capacity_needed[(capacity_needed['Pathway'] == pathways[p]) & (capacity_needed['Work_Sector'] == w)]['Adj_Capacity_Daily_Per_Machine'].iloc[0]

# pathway demand
demand_dic = {} 

for p in pw_count:
    demand_dic[p] = demand_pw_exists[demand_pw_exists['PW'] == pathways[p]][demand_col].iloc[0]

# machines in sector
sector_dic = {}

for s in sectors_num:
    sector_dic[s] = list(machine_exists[machine_exists['Work_Sector'] == s+1].index)

# sector for each machine
ms_dic = {}

for m in machine_count:
    ms_dic[m] = machine_exists[machine_exists['Machine_Code'] == machine_dic[m]]['Work_Sector'].iloc[0]

# create previous capacity dataframe and previous capacity dic, bottleneck dataframe
prev_cap = np.zeros(len(capacity_needed))
count = 0
for p in pathways:
    for o in capacity_needed[capacity_needed['Pathway'] == p]['Operation_Sequence']:
        if o == 1:
            prev_cap[count] = capacity_needed[(capacity_needed['Pathway'] == p) & (capacity_needed['Operation_Sequence'] == o)]['Adj_Capacity_Daily_Per_Machine'].iloc[0]
            count += 1
            continue

        prev_cap[count] = capacity_needed[(capacity_needed['Pathway'] == p) & (capacity_needed['Operation_Sequence'] == o-1)]['Adj_Capacity_Daily_Per_Machine'].iloc[0]
        count += 1

bottleneck = np.zeros(len(capacity_needed))
count = 0
next_count = 0
for p in pathways:
    next_count = count + len(capacity_needed[capacity_needed['Pathway'] == p])
    bottleneck[count:next_count] = capacity_needed[capacity_needed['Pathway'] == p]['Adj_Capacity_Daily_Per_Machine'].min()
    count = next_count

capacity_needed['Inverse_Capacity'] = 1/capacity_needed['Adj_Capacity_Daily_Per_Machine']

# Capacity Dictionary
inverse_capacity_dic = {} 

for p in pw_count:
    inverse_capacity_dic[p] = {}

    for w in capacity_needed[capacity_needed['Pathway'] == pathways[p]]['Work_Sector'].unique():
        inverse_capacity_dic[p][w] = capacity_needed[(capacity_needed['Pathway'] == pathways[p]) & (capacity_needed['Work_Sector'] == w)]['Inverse_Capacity'].iloc[0]

capacity_needed['Adj_Capacity_Daily_Per_Machine_Last'] = prev_cap
capacity_needed['Adj_Capacity_Daily_Per_Machine_Last_Curr_Min'] = [min(capacity_needed['Adj_Capacity_Daily_Per_Machine_Last'].iloc[i], capacity_needed['Adj_Capacity_Daily_Per_Machine'].iloc[i]) for i in range(len(capacity_needed))]

sticky_min = np.zeros(len(capacity_needed))
sticky_op = np.zeros(len(capacity_needed))
count = 0
for p in pathways:
    for o in capacity_needed[capacity_needed['Pathway'] == p]['Operation_Sequence']:
        if o == 1:
            sticky_min[count] = capacity_needed[(capacity_needed['Pathway'] == p) & (capacity_needed['Operation_Sequence'] == o)]['Adj_Capacity_Daily_Per_Machine_Last_Curr_Min'].iloc[0]
            sticky_op[count] = o
            count += 1
            continue

        sticky_min[count] = min(sticky_min[count-1], capacity_needed[(capacity_needed['Pathway'] == p) & (capacity_needed['Operation_Sequence'] == o)]['Adj_Capacity_Daily_Per_Machine_Last_Curr_Min'].iloc[0])

        if sticky_min[count-1] <= capacity_needed[(capacity_needed['Pathway'] == p) & (capacity_needed['Operation_Sequence'] == o)]['Adj_Capacity_Daily_Per_Machine_Last_Curr_Min'].iloc[0]:
            sticky_op[count] = sticky_op[count-1]
        else:
            sticky_op[count] = o

        count += 1

capacity_needed['Bottleneck'] = sticky_min
capacity_needed['Bottleneck_Op'] = sticky_op
capacity_needed['Overall_Bottleneck'] = bottleneck

op_cap_dic = {}

for p in pw_count:
    op_cap_dic[p] = {}
    for o in capacity_needed[capacity_needed['Pathway'] == pathways[p]]['Operation_Sequence'].unique():
        op_cap_dic[p][o] = capacity_needed[(capacity_needed['Pathway'] == pathways[p]) & (capacity_needed['Operation_Sequence'] == o)]['Bottleneck_Op'].iloc[0]

bottle_capacity_dic = {} 

for p in pw_count:
    bottle_capacity_dic[p] = {}

    for w in capacity_needed[capacity_needed['Pathway'] == pathways[p]]['Work_Sector'].unique():
        bottle_capacity_dic[p][w] = capacity_needed[(capacity_needed['Pathway'] == pathways[p]) & (capacity_needed['Work_Sector'] == w)]['Adj_Capacity_Daily_Per_Machine_Last_Curr_Min'].iloc[0]

pw_op_kind_mach = {}

for p in pw_count:
    pw_op_kind_mach[p] = {}
    for o in capacity_needed[capacity_needed['Pathway'] == pathways[p]]['Operation_Sequence']:
        pw_op_kind_mach[p][o] = capacity_needed[(capacity_needed['Pathway'] == pathways[p]) & (capacity_needed['Operation_Sequence'] == o)]['Operation_Desc'].iloc[0]

op_kind_mach = {}
for i in capacity_needed['Operation_Desc'].unique():
    op_kind_mach[i] = [n for n in machine_exists[machine_exists['Machine_Type'] == i].index]

Using license file /Users/benjaminparsons/gurobi.lic
Academic license - for non-commercial use only - expires 2021-08-09


## DEBUG

In [7]:
machine_exists[machine_exists['Work_Sector'] == 51]

,Machine_Code,Machine_Type,Building,Desc,Machine_Notes,Work_Sector
0,ADB1,ADB,110,NaN,NaN,51.0
1,ADB2,ADB,110,NaN,NaN,51.0
2,ADB3,ADB,110,NaN,NaN,51.0
3,ADB4,ADB,110,NaN,NaN,51.0
4,ADB5,ADB,110,NaN,NaN,51.0


In [8]:
capacity_needed[capacity_needed['Work_Sector']==51]

,Pathway,Operation_Sequence,Operation_Desc,Work_Sector,Compliment,Machine_Count_Per_Sector,BPC,Adj_Capacity_Daily_Per_Machine,Inverse_Capacity,Adj_Capacity_Daily_Per_Machine_Last,Adj_Capacity_Daily_Per_Machine_Last_Curr_Min,Bottleneck,Bottleneck_Op,Overall_Bottleneck
140,FG03,10,ADB,51.0,3.0,5.0,203,13000.0,0.000077,32500.0,13000.0,7995.0,5.0,7995.0
153,FG04,10,ADB,51.0,3.0,5.0,204,13000.0,0.000077,32500.0,13000.0,5330.0,5.0,5330.0
166,FG05,11,Driller,51.0,3.0,5.0,206,6500.0,0.000154,26000.0,6500.0,1950.0,5.0,1950.0
195,FG52,11,ADB,51.0,3.0,5.0,254,9100.0,0.000110,22100.0,9100.0,2145.0,3.0,2145.0


In [9]:
pathways[19]

'FG52'

In [10]:
pw_dic[19][11]

[51.0]

In [11]:
pw_op_kind_mach[19]

{1: 'Header',
 2: 'Pointer',
 3: 'Shaver',
 4: 'Heat Treat',
 5: 'Grinder',
 6: 'Fillet Roller',
 7: 'Thread Roller',
 8: 'Driller',
 9: 'Broacher',
 10: 'Broacher',
 11: 'ADB',
 12: 'Flute Grinder'}

In [12]:
test = {}
for p in pw_count:
    test[p] = {}
    for o in range(len(pw_dic[p].keys())):
        test[p][o+1] = [g for g in op_kind_mach[pw_op_kind_mach[p][o+1]] if g in flatten([sector_dic[x-1] for x in pw_dic[p][o+1]])]

In [13]:
for key, value in test.items():
    for key2, value2 in value.items():
        if value2 == []:
            print(key, key2)

In [14]:
machine_dtls.iloc[flatten([sector_dic[x-1] for x in pw_dic[0][6+1]])]

,Machine_Code,Machine_Type,Building,Desc,Machine_Notes,Work_Sector
94,PW14,Washing,110,NaN,NaN,33.0
96,PW16,Washing,110,NaN,NaN,56.0
98,R16,Thread Roller,110,NaN,NaN,56.0
193,TP5,Pointer,110,3300,NaN,55.0
198,TR102,Thread Roller,110,NaN,NaN,52.0
199,TR103,Thread Roller,135,NaN,NaN,8.0
201,TR110,Thread Roller,135,NaN,NaN,57.0
202,TR112,Thread Roller,135,NaN,NaN,10.0
205,TR12,Thread Roller,135,NaN,NaN,8.0
191,TP3,Pointer,135,3300,NaN,4.0


## ADD OPTIMIZATION VARIABLES AND CONSTRAINTS

In [15]:
# Decision variables
employees = mod1.addVars(len(sectors_num), len(days), vtype = GRB.INTEGER)
machines = mod1.addVars(len(machine_count), len(days), len(pw_count), len(max_operations), vtype = GRB.BINARY)
production = mod1.addVars(len(machine_count), len(days), len(pw_count), len(max_operations))
#aux_prod = mod1.addVars(len(machine_count), len(days), len(pw_count), len(max_operations))
#possible_production = mod1.addVars(len(machine_count), len(days), len(pw_count), len(max_operations))
sum_emp_per_day = mod1.addVars(len(days))
max_sum_emp = mod1.addVars(1)

# Constraints

# there needs to be enough employees running the machines in each sector
machines_in_sectors_dic = {}
for s in sectors_num:
    if len(sector_dic[s]) == 0:
        continue
    machines_in_sectors_dic[s] = {}
    for d in days:
        machines_in_sectors_dic[s][d] = mod1.addConstr(sum([machines[m,d,p,o] for m in sector_dic[s] for p in pw_count for o in max_operations]) * sector_compliments[sector_compliments['Work_Sector'] == s+1]['Inverse_Compliment'].iloc[0] <= employees[s,d])

#machines_in_sectors_dic = {}
#for s in sectors_num:
#    if len(sector_dic[s]) == 0:
#        continue
#    machines_in_sectors_dic[s] = {}
#    for d in days:
#        machines_in_sectors_dic[s][d] = mod1.addConstr(sum([production[m,d,p,o]*inverse_capacity_dic[p][s] for m in sector_dic[s] for p in pw_count for o in max_operations]) * sector_compliments[sector_compliments['Work_Sector'] == s+1]['Inverse_Compliment'].iloc[0] <= employees[s,d])        
        
# a machine can only serve one pathway_operation a day
single_pathway = {}
for m in machine_count:
    single_pathway[m] = {}
    for d in days:
        single_pathway[m][d] = mod1.addConstr(sum([machines[m,d,p,o] for p in pw_count for o in max_operations]) <= 1)

#single_pathway = {}
#for m in machine_count:
#    single_pathway[m] = {}
#    for d in days:
#        single_pathway[m][d] = mod1.addConstr(sum([production[m,d,p,o]*inverse_capacity_dic[p][ms_dic[m]] for p in pw_count for o in max_operations]) <= 1)
        
# order of production sequence must be maintained for daily machine assignemnt. An operation may not produce product if its prior operation has not produced at least the number it plans to produce that day.
# sequence = {}
# for p in pw_count:
#    sequence[p] = {}
#    for o in range(len(pw_dic[p].keys())-1):
#        sequence[p][o] = {}
#        for z in days:
#            sequence[p][o][z] = mod1.addConstr(sum([machines[m,d,p,o] * capacity_dic[p][ms_dic[m]] for d in range(z+1) for m in flatten([sector_dic[x-1] for x in pw_dic[p][o+1]])]) - sum([machines[m,z,p,o] * capacity_dic[p][ms_dic[m]] for m in flatten([sector_dic[x-1] for x in pw_dic[p][o+1]])]) >= sum([machines[m,z,p,o+1] * bottle_capacity_dic[p][ms_dic[m]] for m in flatten([sector_dic[x-1] for x in pw_dic[p][o+2]])]))

# order of production sequence must be maintained for daily machine assignemnt. An operation may not produce product if its prior operation has not produced at least the number it plans to produce that day.
#aux = {}
#for p in pw_count:
#    aux[p] = {}
#    for o in range(len(op_cap_dic[p].keys())-1):
#        aux[p][o] = {}
#        for z in days:
#            aux[p][o][z] = {}
#            for mk in flatten([sector_dic[x-1] for x in pw_dic[p][o+1]]):
#                aux[p][o][z][mk] = mod1.addConstr(aux_prod[mk,z,p,o] == sum([machines[m,d,p,op_cap_dic[p][o+1]-1] * capacity_dic[p][ms_dic[m]] for d in range(z+1) for m in flatten([sector_dic[x-1] for x in pw_dic[p][op_cap_dic[p][o+1]]])]) - sum([machines[m,z,p,op_cap_dic[p][o+1]-1] * capacity_dic[p][ms_dic[m]] for m in flatten([sector_dic[x-1] for x in pw_dic[p][op_cap_dic[p][o+1]]])]))

#poss_cap = {}
#for p in pw_count:
#    poss_cap[p] = {}
#    for o in range(len(op_cap_dic[p].keys())-1):
#        poss_cap[p][o] = {}
#        for z in days:
#            poss_cap[p][o][z] = {}
#            for mk in flatten([sector_dic[x-1] for x in pw_dic[p][o+1]]):
#                poss_cap[p][o][z][mk] = mod1.addConstr(possible_production[mk,z,p,o] == min_([aux_prod[mk,z,p,o], capacity_dic[p][ms_dic[mk]]]))

#sequence = {}
#for p in pw_count:
#    sequence[p] = {}
#    for o in range(len(op_cap_dic[p].keys())-1):
#        sequence[p][o] = {}
#        for z in days:
#            sequence[p][o][z] = mod1.addConstr(sum([machines[m,d,p,op_cap_dic[p][o+1]-1] * capacity_dic[p][ms_dic[m]] for d in range(z+1) for m in flatten([sector_dic[x-1] for x in pw_dic[p][op_cap_dic[p][o+1]]])]) - sum([machines[m,z,p,op_cap_dic[p][o+1]-1] * capacity_dic[p][ms_dic[m]] for m in flatten([sector_dic[x-1] for x in pw_dic[p][op_cap_dic[p][o+1]]])]) >= sum([machines[m,z,p,o+1] * possible_production[m,z,p,o+1] for m in flatten([sector_dic[x-1] for x in pw_dic[p][o+2]])]))

sequence = {}
for p in pw_count:
    sequence[p] = {}
    for o in range(len(pw_dic[p].keys())-1):
        sequence[p][o] = {}
        for z in days:
            sequence[p][o][z] = mod1.addConstr(sum([production[m,d,p,o] for d in range(z+1) for m in flatten([sector_dic[x-1] for x in pw_dic[p][o+1]])]) - sum([production[m,z,p,o] for m in flatten([sector_dic[x-1] for x in pw_dic[p][o+1]])]) >= sum([production[m,d,p,o+1] for d in range(z+1) for m in flatten([sector_dic[x-1] for x in pw_dic[p][o+2]])]))

# machine can't produce more than its capacity

capacity = {}
for p in pw_count:
    capacity[p] = {}
    for o in range(len(pw_dic[p].keys())-1):
        capacity[p][o] = {}
        for d in days:
            capacity[p][o][d] = {}
            for m in flatten([sector_dic[x-1] for x in pw_dic[p][o+1]]):
                capacity[p][o][d][m] = mod1.addConstr(production[m,d,p,o] <= capacity_dic[p][ms_dic[m]])
            
# turn off extraneous machines that cant produce output due to insufficient wip (other minimization efforts may cause this by default, but this is for extra saftey)
zero_default = {}
for p in pw_count:
    zero_default[p] = {}
    for o in max_operations:
        zero_default[p][o] = {}
        for z in days:
            zero_default[p][o][z] = {}
            for m in machine_count:
                zero_default[p][o][z][m] = mod1.addConstr(machines[m,z,p,o] <= production[m,z,p,o])
                
zero_default2 = {}
for p in pw_count:
    zero_default2[p] = {}
    for o in max_operations:
        zero_default2[p][o] = {}
        for z in days:
            zero_default2[p][o][z] = {}
            for m in machine_count:
                zero_default2[p][o][z][m] = mod1.addConstr(machines[m,z,p,o] * production[m,z,p,o] == production[m,z,p,o])

# the number of machines running needs to produce enough output for each pathway they support
# output = {}
# for p in pw_count:
#    output[p] = {}
#    for o in range(len(pw_dic[p].keys())):
#        output[p][o] = mod1.addConstr(sum([machines[m,d,p,o] * possible_production[m,d,p,o] for d in days for m in flatten([sector_dic[x-1] for x in pw_dic[p][o+1]])]) >= demand_dic[p])

output = {}
for p in pw_count:
    output[p] = {}
    for o in range(len(pw_dic[p].keys())):
        output[p][o] = mod1.addConstr(sum([production[m,d,p,o] for d in days for m in flatten([sector_dic[x-1] for x in pw_dic[p][o+1]])]) == demand_dic[p])        
               
        
# only proper sectors can support a certain operations
reduce_sector = {}
for p in pw_count:
    reduce_sector[p] = {}
    for o in range(len(pw_dic[p].keys())):
        reduce_sector[p][o] = mod1.addConstr(sum([machines[m,d,p,o] for d in days for m in [i for i in machine_count if i not in flatten([sector_dic[x-1] for x in pw_dic[p][o+1]])]]) == 0)

# only proper machines in sectors can support certain operations
#reduce_machine = {}
#for p in pw_count:
#    reduce_machine[p] = {}
#    for o in range(len(pw_dic[p].keys())):
#        reduce_machine[p][o] = mod1.addConstr(sum([machines[m,d,p,o] for d in days for m in [i for i in machine_count if i not in [g for g in op_kind_mach[pw_op_kind_mach[p][o+1]] if g in flatten([sector_dic[x-1] for x in pw_dic[p][o+1]])]]]) == 0)

reduce_product = {}
for p in pw_count:
    reduce_product[p] = {}
    for o in range(len(pw_dic[p].keys())):
        reduce_product[p][o] = {}
        for d in days:
            reduce_product[p][o][d] = {}
            for m in [i for i in machine_count if i not in [g for g in op_kind_mach[pw_op_kind_mach[p][o+1]] if g in flatten([sector_dic[x-1] for x in pw_dic[p][o+1]])]]:
                reduce_product[p][o][d][m] = mod1.addConstr(production[m,d,p,o] == 0)

reduce_machine = {}
for p in pw_count:
    reduce_machine[p] = {}
    temp = [x for x in max_operations if x not in [i for i in range(len(pw_dic[p].keys()))]]
    for o in temp:
        reduce_machine[p][o] = mod1.addConstr(sum([production[m,d,p,o] for d in days for m in machine_count]) == 0)


# set only two machines to a pathway per day
two_pathway = {}
for p in pw_count:
    two_pathway[p] = {}
    for d in days:
        two_pathway[p][d] = {}
        for o in range(len(pw_dic[p].keys())):
            two_pathway[p][d][o] = mod1.addConstr(sum([machines[m,d,p,o] for m in machine_count]) <= 2)
        
# number of employees used each day needs to be the sum of the employees in all sectors
emp_per_day_dic = {}
for d in days:
    emp_per_day_dic[d] = mod1.addConstr(sum([employees[s,d] for s in sectors_num]) == sum_emp_per_day[d])

m_ax = {}
for d in days:
    m_ax[d] = mod1.addConstr(max_sum_emp[0] >= sum_emp_per_day[d])

# Set objective function
mod1.setObjective(max_sum_emp[0], GRB.MINIMIZE)

In [16]:
# set a max numbner of machines per pathway a day (no more than 2) - done
# half shifts throughout the day
# longer periods of time so that we dont have a boom and bust as if the factory was opening and closing from nothing
# start of the month WIP distribution
# user friendly inputs
# simulating machine drop offs
# simulating labor availability fluctuations 
# simulating production failure
# make sure that pw are not split between different work sectors

## OPTIMIZE

In [17]:
# Update and solve
mod1.setParam(GRB.Param.Method, 3)
mod1.setParam(GRB.Param.NodeMethod, 2)
mod1.setParam(GRB.Param.Presolve, 2)
mod1.setParam(GRB.Param.MIPFocus, 1)
mod1.update()
mod1.optimize()

Changed value of parameter Method to 3
   Prev: -1  Min: -1  Max: 5  Default: -1
Changed value of parameter NodeMethod to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter Presolve to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter MIPFocus to 1
   Prev: 0  Min: 0  Max: 3  Default: 0
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 5225587 rows, 6351001 columns and 21873975 nonzeros
Model fingerprint: 0x897b91df
Model has 3174600 quadratic constraints
Variable types: 3174631 continuous, 3176370 integer (3174600 binary)
Coefficient statistics:
  Matrix range     [2e-01, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+06]
Presolve removed 0 rows and 0 columns (presolve time = 6s) ...
Presolve removed 3979627 rows and 0 

    39    58   13.00000   11 1809   15.00000   13.00000  13.3% 81923 1520s
    57   107   13.00000   15 1740   15.00000   13.00000  13.3% 80964 1807s
   106   208   13.00000   27 1765   15.00000   13.00000  13.3% 79806 2465s
   207   403   13.00000   52 1817   15.00000   13.00000  13.3% 78724 3722s
   403   538   13.00000  101 1525   15.00000   13.00000  13.3% 77189 5085s
   573   698   13.00084  144 1170   15.00000   13.00000  13.3% 72337 6507s
   764   831   14.00000  182 1373   15.00000   13.00000  13.3% 70389 8044s
   949   870     cutoff  238        15.00000   13.00000  13.3% 70585 9655s
  1097  1103   14.00000  122 1699   15.00000   13.00000  13.3% 69715 11412s
H 1395   668                      14.0000000   13.00000  7.14% 69240 13219s
  1444   668     cutoff  210        14.00000   13.00000  7.14% 69095 13220s
  1625   457   13.00000  108 20066   14.00000   13.00000  7.14% 69353 13258s
  1627   458   13.00000   14 1204   14.00000   13.00000  7.14% 69268 13297s
  1628   459   13.0

In [18]:
mod1.objval

14.0

In [19]:
machine_num = [i[0] for i in production if production[i].x >= 1]
solution_machines = [machine_dic[i[0]] for i in production if production[i].x >= 1]
soltion_days = [i[1]+1 for i in production if production[i].x >= 1]
solution_pw = [pathways[i[2]] for i in production if production[i].x >= 1]
solution_operation = [i[3]+1 for i in production if production[i].x >= 1]
solution_desc = [pw_op_kind_mach[i[2]][i[3]+1] for i in production if production[i].x >= 1]
solution_sector = [ms_dic[i[0]] for i in production if production[i].x >= 1]
solution_produced = [production[i].x for i in production if production[i].x >= 1]

In [20]:
pd.options.display.max_rows = 999
active = pd.DataFrame({'Machine_Num': machine_num, 'Machine': solution_machines, 'Day': soltion_days, 'PW': solution_pw, 'Operation': solution_operation, 'Description': solution_desc, 'Sector': solution_sector, 'Produced': solution_produced}).sort_values(by=['Day'])

work_array = np.zeros([len(sectors_num), len(days)])

for s in sectors_num:
    work_array[s,:] = [employees[s,d].x for d in days]
    
emp_schedule = pd.DataFrame(work_array, index = [f'Sector {i+1}' for i in sectors_num], columns = [f'Day {i+1}' for i in days])
emp_schedule = emp_schedule.round()

labor_mix_dic = {}

for d in days:
    labor_mix_dic[d+1] = {}
    if len(active[active['Day'] == d+1]['Sector'].unique()) == 0:
        continue
    for s in active[active['Day'] == d+1]['Sector'].unique():
        if len(active[(active['Day'] == d+1) & (active['Sector'] == s)]['Description'].unique()) > 1:
            for desc in active[(active['Day'] == d+1) & (active['Sector'] == s)]['Description'].unique():
                labor_mix_dic[d+1][desc] = len(active[(active['Day'] == d+1) & (active['Sector'] == s) & (active['Description'] == desc)])
        else:
            labor_mix_dic[d+1][active[(active['Day'] == d+1) & (active['Sector'] == s)]['Description'].iloc[0]] = work_array[int(s)-1,d]


In [21]:
labor_req = pd.DataFrame(labor_mix_dic).fillna(0)
labor_req['Max'] = labor_req.max(axis = 1)
labor_req.loc['Sum'] = labor_req.sum()
labor_req

,1,2,3,4,5,6,7,8,9,10,...,22,23,24,25,26,27,28,29,30,Max
Header,14.0,8.0,2.0,2.0,0.0,3.0,4.0,2.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0
CNC,0.0,1.0,2.0,2.0,1.0,1.0,0.0,1.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
Heat Treat,0.0,2.0,1.0,1.0,3.0,3.0,4.0,1.0,4.0,1.0,...,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,5.0
Pointer,0.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
Shaver,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Pointer-Lathe,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Trimmer,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Stamper,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
Grinder,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,0.0,2.0
Fillet Roller,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,4.0,1.0,1.0,0.0,4.0


In [22]:
emp_schedule

,Day 1,Day 2,Day 3,Day 4,Day 5,Day 6,Day 7,Day 8,Day 9,Day 10,...,Day 21,Day 22,Day 23,Day 24,Day 25,Day 26,Day 27,Day 28,Day 29,Day 30
Sector 1,14.0,8.0,2.0,2.0,0.0,3.0,4.0,2.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Sector 2,-0.0,-0.0,1.0,-0.0,-0.0,1.0,1.0,0.0,-0.0,1.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
Sector 3,-0.0,-0.0,-0.0,1.0,-0.0,-0.0,-0.0,-0.0,1.0,-0.0,...,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
Sector 4,-0.0,1.0,1.0,1.0,-0.0,0.0,1.0,1.0,0.0,-0.0,...,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0
Sector 5,-0.0,-0.0,-0.0,-0.0,2.0,-0.0,-0.0,1.0,-0.0,-0.0,...,1.0,0.0,1.0,2.0,1.0,2.0,1.0,0.0,-0.0,-0.0
Sector 6,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,...,0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,0.0,0.0,-0.0
Sector 7,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,...,-0.0,1.0,0.0,1.0,2.0,1.0,4.0,2.0,3.0,-0.0
Sector 8,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,...,-0.0,-0.0,0.0,-0.0,-0.0,0.0,1.0,-0.0,1.0,5.0
Sector 9,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0
Sector 10,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,...,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0


In [23]:
sum_prod_dic = {}
for p in pathways:
    sum_prod_dic[p] = {}
    for d in days:
        sum_prod_dic[p][d+1] = {}
        for o in capacity_needed[capacity_needed['Pathway'] == p]['Operation_Sequence'].unique():
            sum_prod_dic[p][d+1][o] = 0


for p in pw_count:
    for d in days:
        for o in capacity_needed[capacity_needed['Pathway'] == pathways[p]]['Operation_Sequence'].unique():
            if len(active[(active['Day'] == d+1) & (active['PW'] == pathways[p]) & (active['Operation'] == o)]) == 0:
                if d == 0:
                    continue
                else:
                    sum_prod_dic[pathways[p]][d+1][o] = sum_prod_dic[pathways[p]][d][o]
            else:
                if d == 0:
                    sum_prod_dic[pathways[p]][d+1][o] += sum([production[m,d,p,o-1].x for m in active[(active['Day'] == d+1) & (active['PW'] == pathways[p]) & (active['Operation'] == o)]['Machine_Num']])
                else:
                    sum_prod_dic[pathways[p]][d+1][o] += sum([production[m,d,p,o-1].x for m in active[(active['Day'] == d+1) & (active['PW'] == pathways[p]) & (active['Operation'] == o)]['Machine_Num']]) + sum_prod_dic[pathways[p]][d][o]

In [24]:
pd.options.display.max_columns = 999

with pd.ExcelWriter('Past_Due_and_Current.xlsx') as writer:  

    pd.DataFrame(pd.DataFrame(demand_pw_exists[['PW','Past_Due_and_Current']])).to_excel(writer, sheet_name = 'Demand')
    
    labor_req.to_excel(writer, sheet_name = 'Labor_Requirement')

    for p in pathways:
        active[active['PW'] == p].to_excel(writer, sheet_name = p)

        pd.DataFrame(sum_prod_dic[p]).to_excel(writer, sheet_name = p, startcol = len(active[active['PW'] == p].columns)+2)

In [25]:
work_array.max(axis=1).sum()

85.0